In [68]:
# ライブラリimport
import argparse
import logging
import os

import pandas as pd
from pandas import DataFrame
import pandas.tseries.offsets as offsets
import numpy as np
from datetime import date, timedelta, time, datetime
from tabulate import tabulate

from itertools import product

from ailab_tools.smn import ImpalaResource
from ailab_tools.utils import GMail, GSheet

from collections import OrderedDict

import zipfile
import glob

import gc

from ailab_tools.magics import slack_notify

In [69]:
start_date = '2019-11-08'
end_date = '2019-11-10'

#クロスデバイス売上集計スプレッドシート
gsheet_url = "https://docs.google.com/spreadsheets/d/1od23SIxDx1qtM2LcVNosvFdJ9eDBNGRp2dTIZ9cLxsE/edit#gid=0"

# 全体集計（速報）

In [70]:
query = (
    f"""
    select
    concat_ws('-',year,month,day) as yymmdd
    ,ads.advertiser_id
    ,hie.advertiser_name
    ,agu1.unit
    ,ads.strategy_id
    ,hie.strategy_name
    ,seg.segment_names as segment_name
    ,ad_type
    ,case when ad_type = 0 then 'RTG'
    when ad_type = 1 then 'ATA'
    when ad_type = 2 then 'カスタム'
    when ad_type = 3 then '類似'
    when ad_type = 4 then 'DYC'
    else 'Unknown'
    end as ad_type_name
    ,ctgr.name as product_ctgr_name
    ,conc.name as product_ctgr_concrete_name
    ,sum(case when action_type = "I" then 1 else 0 end) imp
    ,sum(case when action_type = "C" then 1 else 0 end) click
    ,sum(case when action_type = "CTV1" then 1 else 0 end) ctv1
    ,sum(case when action_type = "CTV2" then 1 else 0 end) ctv2
    ,sum(case when action_type in("CTV1","CTV2") then 1 else 0 end) ctv
    ,sum(case when action_type = "VTV1" then 1 else 0 end) vtv1
    ,sum(case when action_type = "VTV2" then 1 else 0 end) vtv2
    ,sum(case when action_type in("VTV1","VTV2") then 1 else 0 end) vtv
    ,sum(winning_price)/1000000 as win_price
    ,sum(winning_price*(1+cast(csm.margin as float)/100))/1000000 net_sales
    ,sum(winning_price*(1+cast(csm.margin as float)/100)/(1-cast(cam.margin as float)/100))/1000000 gross_sales
    from dsplog.adserver ads 
    inner join dm.hierarchies hie on ads.target_id = hie.target_id 
    left join agency_console.campaign cp on ads.strategy_id = cp.real_strategy_id 
    left join agency_console.campaign_agency_margin cam on cp.campaign_id = cam.campaign_id and concat_ws('-', year, month, day) between cam.start_date and cam.end_date 
    left join agency_console.campaign_smn_margin csm on cp.campaign_id = csm.campaign_id and concat_ws('-', year, month, day) between csm.start_date and csm.end_date 
    left join agency_console.advertiser ad on cp.advertiser_id = ad.advertiser_id
    left join agency_console.agency_user agu1 on ad.smn_sales_member_id = agu1.agency_user_id 
    left join agency_console.agency_user agu2 on ad.smn_operation_member_id = agu2.agency_user_id
    left join (select target_id,group_concat(segment_name,'|') as segment_names from dm.segment_hierarchies group by target_id) seg on ads.target_id = seg.target_id
    left join agency_console.product_ctgr ctgr on hie.product_ctgr_id = ctgr.product_ctgr_id
    left join agency_console.product_ctgr_concrete conc on hie.product_ctgr_concrete_id = conc.product_ctgr_concrete_id
    where 
    concat_ws('-', year, month, day) between '{start_date}' and  '{end_date}'
    and (segment_names like '%XD%'
    or tactics_name like '%XD%')
    group by 1,2,3,4,5,6,7,8,9,10,11
    order by yymmdd
    """
    )

with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    df_shukei = ir.sql_to_pandas(query)
    df_shukei = df_shukei.astype({'win_price':float, 'net_sales':float, 'gross_sales':float})
    gs = GSheet()
    sheet_index = 0
    col_offset=0
    gs.append_data(gsheet_url, sheet_index, df_shukei, col_offset)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
    s

XDリタゲグループ登録済みかどうかのチェック

In [4]:
query = (
    f"""
    select
    hie.advertiser_id
    ,hie.advertiser_name
    /*,agu1.name as smn_sales_name
    ,agu2.name as smn_operation_name*/
    ,agu1.unit
    ,count(distinct rtg.name) as n
    from dm.hierarchies hie
    inner join agency_console.campaign cp on hie.strategy_id = cp.real_strategy_id
    inner join agency_console.advertiser ad on cp.advertiser_id = ad.advertiser_id
    inner join agency_console.agency_user agu1 on ad.smn_sales_member_id = agu1.agency_user_id 
    inner join agency_console.agency_user agu2 on ad.smn_operation_member_id = agu2.agency_user_id
    inner join console.retargeting_group rtg on hie.advertiser_id = rtg.advertiser_id
    where 
    rtg.name like '%XD_%'
    and rtg.id not in (106250,106391,106430)
    group by 1,2,3
    """
    )

with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    df_rtg = ir.sql_to_pandas(query)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
    s

In [5]:
df_rtg = df_rtg.sort_values(by='advertiser_id')
df_rtg = df_rtg.drop_duplicates(['advertiser_id'])
df_rtg.head()

,advertiser_id,advertiser_name,unit,n
123,46,富士通株式会社,東日本1課,2
220,53,スカパーJSAT株式会社,東日本1課,3
37,62,株式会社日本サイバー教育研究所,東日本2課,4
153,194,株式会社オーネット,東日本2課,2
62,353,(S)株式会社やずや,西日本九州,3


In [6]:
#クロスデバイス売上集計スプレッドシート
#シート上のデータ消去
for i in range(1,2):
    gs = GSheet()
    gs.clear_sheets(gsheet_url, [i])

INFO:ailab_tools.utils.gsheet:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.utils.gsuite:created OAuth2 credentials.
INFO:googleapiclient.discovery:URL being requested: GET https://www.googleapis.com/discovery/v1/apis/sheets/v4/rest
INFO:googleapiclient.discovery:URL being requested: GET https://sheets.googleapis.com/v4/spreadsheets/1od23SIxDx1qtM2LcVNosvFdJ9eDBNGRp2dTIZ9cLxsE?alt=json
INFO:googleapiclient.discovery:URL being requested: POST https://sheets.googleapis.com/v4/spreadsheets/1od23SIxDx1qtM2LcVNosvFdJ9eDBNGRp2dTIZ9cLxsE/values/rtg%21A1%3AZ1002:clear?alt=json


In [7]:
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    gs = GSheet()
    gs.add_data(gsheet_url, 1, df_rtg, [0, 0], True)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:ailab_tools.utils.gsheet:loa

In [8]:
df_rtg.shape[0]

212

In [9]:
#クロスデバイス・リタゲ優先導入ランク一覧
gsheet_url2 = "https://docs.google.com/spreadsheets/d/1NgSpXDCRqWN_Omw8NmIl03_c5bWBynV4fhVRD_Yo7No/edit#gid=1364599456"

In [10]:
#クロスデバイス・リタゲ優先導入ランク一覧
#シート上のデータ消去
for i in range(8,9):
    gs = GSheet()
    gs.clear_sheets(gsheet_url2, [i])

INFO:ailab_tools.utils.gsheet:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.utils.gsuite:created OAuth2 credentials.
INFO:googleapiclient.discovery:URL being requested: GET https://www.googleapis.com/discovery/v1/apis/sheets/v4/rest
INFO:googleapiclient.discovery:URL being requested: GET https://sheets.googleapis.com/v4/spreadsheets/1NgSpXDCRqWN_Omw8NmIl03_c5bWBynV4fhVRD_Yo7No?alt=json
INFO:googleapiclient.discovery:URL being requested: POST https://sheets.googleapis.com/v4/spreadsheets/1NgSpXDCRqWN_Omw8NmIl03_c5bWBynV4fhVRD_Yo7No/values/8_XD%20rtg%E7%99%BB%E9%8C%B2%E6%95%B0%EF%BC%88%E6%9B%B4%E6%96%B0%E7%94%A8%EF%BC%89%21A1%3AAB1001:clear?alt=json


In [11]:
#クロスデバイス・リタゲ優先導入ランク一覧（8_XD rtg登録数（自動更新））
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    gs = GSheet()
    gs.add_data(gsheet_url2, 8, df_rtg, [0, 0], True)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:ailab_tools.utils.gsheet:loa

クロスデバイス売上集計データを読込、リタゲ導入済み案件をピックアップ

In [12]:
gs = GSheet()
sheet_df = gs.sheetdata_to_dataframe(gsheet_url)[0]
rtg_df = sheet_df[(sheet_df['ad_type_name'] == "RTG")|(sheet_df['ad_type_name'] == "DYC")]
rtg_df.head()

INFO:ailab_tools.utils.gsheet:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.utils.gsuite:created OAuth2 credentials.
INFO:googleapiclient.discovery:URL being requested: GET https://www.googleapis.com/discovery/v1/apis/sheets/v4/rest
INFO:googleapiclient.discovery:URL being requested: GET https://sheets.googleapis.com/v4/spreadsheets/1od23SIxDx1qtM2LcVNosvFdJ9eDBNGRp2dTIZ9cLxsE?alt=json
INFO:googleapiclient.discovery:URL being requested: GET https://sheets.googleapis.com/v4/spreadsheets/1od23SIxDx1qtM2LcVNosvFdJ9eDBNGRp2dTIZ9cLxsE/values/%E5%85%A8%E4%BD%93%21A1%3AAE47425?valueRenderOption=FORMATTED_VALUE&alt=json


,yymmdd,device,advertiser_id,advertiser_name,smn_sales_name,smn_operation_name,unit,strategy_id,strategy_name,segment_name,...,ctv,vtv1,vtv2,vtv,win_price,net_sales,gross_sales,V拡張適用フラグ,ad_type_name2,年月
56,2019-02-14,PC,7410,(S)株式会社マイナビ（マイナビバイト）,田村太征,大場優可理,東日本1課,42442,(S)ゼンショーベストクルー_RT_ALL,XD_ブログウォッチャー1902,...,0,0,0,0,NaN,NaN,NaN,NaN,RTG,2019-02
58,2019-02-14,SP,7410,(S)株式会社マイナビ（マイナビバイト）,田村太征,大場優可理,東日本1課,42442,(S)ゼンショーベストクルー_RT_ALL,XD_ブログウォッチャー1902,...,0,0,0,0,1.0,2.0,3.0,NaN,RTG,2019-02
66,2019-02-15,SP,7410,(S)株式会社マイナビ（マイナビバイト）,田村太征,大場優可理,東日本1課,42442,(S)ゼンショーベストクルー_RT_ALL,XD_ブログウォッチャー1902,...,0,0,0,0,51.0,87.0,109.0,NaN,RTG,2019-02
72,2019-02-15,SP,7410,(S)株式会社マイナビ（マイナビバイト）,田村太征,大場優可理,東日本1課,42444,(S)ゼンショーベストクルー_RT_飲食,XD_ブログウォッチャー1902,...,0,0,0,0,3.0,4.0,5.0,NaN,RTG,2019-02
75,2019-02-15,PC,7410,(S)株式会社マイナビ（マイナビバイト）,田村太征,大場優可理,東日本1課,42442,(S)ゼンショーベストクルー_RT_ALL,XD_ブログウォッチャー1902,...,0,0,0,0,1.0,2.0,2.0,NaN,RTG,2019-02


In [13]:
import datetime as dt

In [14]:
rtg_df['yymmdd'] = pd.to_datetime(rtg_df['yymmdd'])

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [15]:
rtg_df = rtg_df[rtg_df['yymmdd'] > dt.datetime(2019,5,20)]
rtg_df = rtg_df.sort_values(['advertiser_id','yymmdd']) 
rtg_df = rtg_df.drop_duplicates(['advertiser_id'])
rtg_df = rtg_df[['advertiser_id','advertiser_name','ad_type_name','yymmdd']]
rtg_df['yymmdd'] = rtg_df['yymmdd'].dt.strftime("%Y/%m/%d")
rtg_df.head()

,advertiser_id,advertiser_name,ad_type_name,yymmdd
41678,46,富士通株式会社,RTG,2019/09/11
11260,53,スカパーJSAT株式会社,DYC,2019/06/13
15800,62,株式会社日本サイバー教育研究所,RTG,2019/06/27
13121,194,株式会社オーネット,RTG,2019/06/19
10497,353,(S)株式会社やずや,RTG,2019/06/10


In [16]:
rtg_df.shape[0]

132

In [17]:
#クロスデバイス・リタゲ優先導入ランク一覧
#シート上のデータ消去
for i in range(9,10):
    gs = GSheet()
    gs.clear_sheets(gsheet_url2, [i])

INFO:ailab_tools.utils.gsheet:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.utils.gsuite:created OAuth2 credentials.
INFO:googleapiclient.discovery:URL being requested: GET https://www.googleapis.com/discovery/v1/apis/sheets/v4/rest
INFO:googleapiclient.discovery:URL being requested: GET https://sheets.googleapis.com/v4/spreadsheets/1NgSpXDCRqWN_Omw8NmIl03_c5bWBynV4fhVRD_Yo7No?alt=json
INFO:googleapiclient.discovery:URL being requested: POST https://sheets.googleapis.com/v4/spreadsheets/1NgSpXDCRqWN_Omw8NmIl03_c5bWBynV4fhVRD_Yo7No/values/9_%E5%B0%8E%E5%85%A5%E6%A1%88%E4%BB%B6%EF%BC%88%E6%9B%B4%E6%96%B0%E7%94%A8%EF%BC%89%21A1%3AAA1001:clear?alt=json


In [18]:
#クロスデバイス・リタゲ優先導入ランク一覧へ書込み（9_導入案件（自動更新））
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    gs = GSheet()
    gs.add_data(gsheet_url2, 9, rtg_df, [0, 0], True)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:ailab_tools.utils.gsheet:loa

start_date = '2019-08-06'
end_date = '2019-08-06'

# 効果測定
考え方①<br>
クロスデイバス・リタゲ配信が同居しているキャンペーンを抽出して全体を集計する

In [19]:
%%slack_notify @yusuke_akada
query = (
        f"""
        with xd_tmp as (
        select
        'XD' as castegory
        ,concat_ws('-',year,month,day) as yymmdd
        ,ads.advertiser_id
        ,hie.advertiser_name
        ,agu1.name as smn_sales_name
        ,agu2.name as smn_operation_name
        ,agu1.unit
        ,ads.strategy_id
        ,hie.strategy_name
        ,seg.segment_name
        ,ad_type
        ,case when ad_type = 0 then 'RTG'
        when ad_type = 1 then 'ATA'
        when ad_type = 2 then 'カスタム'
        when ad_type = 3 then '類似'
        when ad_type = 4 then 'DYC'
        else 'Unknown'
        end as ad_type_name
        ,hie.product_ctgr_id
        ,ctgr.name as product_ctgr_name
        ,hie.product_ctgr_concrete_id
        ,conc.name as product_ctgr_concrete_name
        ,sum(case when action_type = "I" then 1 else 0 end) imp
        ,sum(case when action_type = "C" then 1 else 0 end) click
        ,sum(case when action_type = "CTV1" then 1 else 0 end) ctv1
        ,sum(case when action_type = "CTV2" then 1 else 0 end) ctv2
        ,sum(case when action_type in("CTV1","CTV2") then 1 else 0 end) ctv
        ,sum(case when action_type = "VTV1" then 1 else 0 end) vtv1
        ,sum(case when action_type = "VTV2" then 1 else 0 end) vtv2
        ,sum(case when action_type in("VTV1","VTV2") then 1 else 0 end) vtv
        ,sum(winning_price)/1000000 as win_price
        ,sum(winning_price*(1+csm.margin/100))/1000000 net_sales
        ,sum(winning_price*(1+csm.margin/100)/(1-cam.margin/100))/1000000 gross_sales
        ,sum(case when xd_action_type in("xCTV1","xCTV2") then 1 else 0 end) xctv
        ,sum(case when xd_action_type in("xVTV1","xVTV2") then 1 else 0 end) xvtv
        from dsplog.adserver ads 
        inner join dm.hierarchies hie on ads.target_id = hie.target_id 
        left join agency_console.campaign cp on ads.strategy_id = cp.real_strategy_id 
        left join agency_console.campaign_agency_margin cam on cp.campaign_id = cam.campaign_id and concat_ws('-', year, month, day) between cam.start_date and cam.end_date 
        left join agency_console.campaign_smn_margin csm on cp.campaign_id = csm.campaign_id and concat_ws('-', year, month, day) between csm.start_date and csm.end_date 
        left join agency_console.advertiser ad on cp.advertiser_id = ad.advertiser_id
        left join agency_console.agency_user agu1 on ad.smn_sales_member_id = agu1.agency_user_id 
        left join agency_console.agency_user agu2 on ad.smn_operation_member_id = agu2.agency_user_id
        left join dm.segment_hierarchies seg on ads.target_id = seg.target_id
        left join agency_console.product_ctgr ctgr on hie.product_ctgr_id = ctgr.product_ctgr_id
        left join agency_console.product_ctgr_concrete conc on hie.product_ctgr_concrete_id = conc.product_ctgr_concrete_id
        where 
        concat_ws('-', year, month, day) between '{start_date}' and  '{end_date}' 
        and (segment_name like '%XD%'
        or tactics_name like '%XD%')
        and ad_type in (0,4)
        group by 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
        order by yymmdd
        )
        ,
        all_tmp as (
        select
        'ALL' as category
        ,concat_ws('-',year,month,day) as yymmdd
        ,ads.advertiser_id
        ,hie.advertiser_name
        ,agu1.name as smn_sales_name
        ,agu2.name as smn_operation_name
        ,agu1.unit
        ,ads.strategy_id
        ,hie.strategy_name
        ,"" as segment_name
        ,hie.ad_type
        ,case when hie.ad_type = 0 then 'RTG'
        when hie.ad_type = 1 then 'ATA'
        when hie.ad_type = 2 then 'カスタム'
        when hie.ad_type = 3 then '類似'
        when hie.ad_type = 4 then 'DYC'
        else 'Unknown'
        end as ad_type_name
        ,hie.product_ctgr_id
        ,ctgr.name as product_ctgr_name
        ,hie.product_ctgr_concrete_id
        ,conc.name as product_ctgr_concrete_name
        ,sum(case when action_type = "I" then 1 else 0 end) imp
        ,sum(case when action_type = "C" then 1 else 0 end) click
        ,sum(case when action_type = "CTV1" then 1 else 0 end) ctv1
        ,sum(case when action_type = "CTV2" then 1 else 0 end) ctv2
        ,sum(case when action_type in("CTV1","CTV2") then 1 else 0 end) ctv
        ,sum(case when action_type = "VTV1" then 1 else 0 end) vtv1
        ,sum(case when action_type = "VTV2" then 1 else 0 end) vtv2
        ,sum(case when action_type in("VTV1","VTV2") then 1 else 0 end) vtv
        ,sum(winning_price)/1000000 as win_price
        ,sum(winning_price*(1+csm.margin/100))/1000000 net_sales
        ,sum(winning_price*(1+csm.margin/100)/(1-cam.margin/100))/1000000 gross_sales
        ,sum(case when xd_action_type in("xCTV1","xCTV2") then 1 else 0 end) xctv
        ,sum(case when xd_action_type in("xVTV1","xVTV2") then 1 else 0 end) xvtv
        from dsplog.adserver ads 
        inner join dm.hierarchies hie on ads.tactics_id = hie.tactics_id 
        left join agency_console.campaign cp on ads.strategy_id = cp.real_strategy_id 
        left join agency_console.campaign_agency_margin cam on cp.campaign_id = cam.campaign_id and concat_ws('-', year, month, day) between cam.start_date and cam.end_date 
        left join agency_console.campaign_smn_margin csm on cp.campaign_id = csm.campaign_id and concat_ws('-', year, month, day) between csm.start_date and csm.end_date 
        left join agency_console.advertiser ad on cp.advertiser_id = ad.advertiser_id
        left join agency_console.agency_user agu1 on ad.smn_sales_member_id = agu1.agency_user_id 
        left join agency_console.agency_user agu2 on ad.smn_operation_member_id = agu2.agency_user_id
        left join agency_console.product_ctgr ctgr on hie.product_ctgr_id = ctgr.product_ctgr_id
        left join agency_console.product_ctgr_concrete conc on hie.product_ctgr_concrete_id = conc.product_ctgr_concrete_id
        inner join xd_tmp on xd_tmp.strategy_id = ads.strategy_id
        where 
        concat_ws('-', year, month, day) between '{start_date}' and  '{end_date}' 
        and hie.ad_type in (0,4)
        group by 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
        order by yymmdd
        )
        select *
        from xd_tmp
        
        union all
        
        select *
        from all_tmp
        """
)
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    df_shukei2 = ir.sql_to_pandas(query)
    df_shukei2 = df_shukei2.astype({'win_price':float, 'net_sales':float, 'gross_sales':float})
    gs = GSheet()
    url = "https://docs.google.com/spreadsheets/d/1ayT6UVygczD8b1Q_XkSsdg1KskH4uCYTp_pyrSdCAZ4/edit#gid=865724583"
    sheet_index = 0
    col_offset=0
    gs.append_data(url, sheet_index, df_shukei2, col_offset)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
     

# 効果測定
考え方②<br>
クロスデイバス・リタゲ配信を実施している広告主全体を抽出して集計する

In [20]:
%%slack_notify @yusuke_akada
query = (
        f"""
        with xd_tmp as (
        select
        'XD' as castegory
        ,concat_ws('-',year,month,day) as yymmdd
        ,ads.advertiser_id
        ,hie.advertiser_name
        ,agu1.name as smn_sales_name
        ,agu2.name as smn_operation_name
        ,agu1.unit
        ,ad_type
        ,case when ad_type = 0 then 'RTG'
        when ad_type = 1 then 'ATA'
        when ad_type = 2 then 'カスタム'
        when ad_type = 3 then '類似'
        when ad_type = 4 then 'DYC'
        else 'Unknown'
        end as ad_type_name
        ,hie.product_ctgr_id
        ,ctgr.name as product_ctgr_name
        ,sum(case when action_type = "I" then 1 else 0 end) imp
        ,sum(case when action_type = "C" then 1 else 0 end) click
        ,sum(case when action_type = "CTV1" then 1 else 0 end) ctv1
        ,sum(case when action_type = "CTV2" then 1 else 0 end) ctv2
        ,sum(case when action_type in("CTV1","CTV2") then 1 else 0 end) ctv
        ,sum(case when action_type = "VTV1" then 1 else 0 end) vtv1
        ,sum(case when action_type = "VTV2" then 1 else 0 end) vtv2
        ,sum(case when action_type in("VTV1","VTV2") then 1 else 0 end) vtv
        ,sum(winning_price)/1000000 as win_price
        ,sum(winning_price*(1+csm.margin/100))/1000000 net_sales
        ,sum(winning_price*(1+csm.margin/100)/(1-cam.margin/100))/1000000 gross_sales
        ,sum(case when xd_action_type in("xCTV1","xCTV2") then 1 else 0 end) xctv
        ,sum(case when xd_action_type in("xVTV1","xVTV2") then 1 else 0 end) xvtv
        from dsplog.adserver ads 
        inner join dm.hierarchies hie on ads.target_id = hie.target_id 
        left join agency_console.campaign cp on ads.strategy_id = cp.real_strategy_id 
        left join agency_console.campaign_agency_margin cam on cp.campaign_id = cam.campaign_id and concat_ws('-', year, month, day) between cam.start_date and cam.end_date 
        left join agency_console.campaign_smn_margin csm on cp.campaign_id = csm.campaign_id and concat_ws('-', year, month, day) between csm.start_date and csm.end_date 
        left join agency_console.advertiser ad on cp.advertiser_id = ad.advertiser_id
        left join agency_console.agency_user agu1 on ad.smn_sales_member_id = agu1.agency_user_id 
        left join agency_console.agency_user agu2 on ad.smn_operation_member_id = agu2.agency_user_id
        left join dm.segment_hierarchies seg on ads.target_id = seg.target_id
        left join agency_console.product_ctgr ctgr on hie.product_ctgr_id = ctgr.product_ctgr_id
        left join agency_console.product_ctgr_concrete conc on hie.product_ctgr_concrete_id = conc.product_ctgr_concrete_id
        where 
        concat_ws('-', year, month, day) between '{start_date}' and  '{end_date}' 
        and (segment_name like '%XD%'
        or tactics_name like '%XD%')
        and hie.ad_type in (0,4)
        group by 1,2,3,4,5,6,7,8,9,10,11
        order by yymmdd
        )
        ,
        all_tmp as (
        select
        'ALL' as category
        ,concat_ws('-',year,month,day) as yymmdd
        ,ads.advertiser_id
        ,hie.advertiser_name
        ,agu1.name as smn_sales_name
        ,agu2.name as smn_operation_name
        ,agu1.unit
        ,hie.ad_type
        ,case when hie.ad_type = 0 then 'RTG'
        when hie.ad_type = 1 then 'ATA'
        when hie.ad_type = 2 then 'カスタム'
        when hie.ad_type = 3 then '類似'
        when hie.ad_type = 4 then 'DYC'
        else 'Unknown'
        end as ad_type_name
        ,hie.product_ctgr_id
        ,ctgr.name as product_ctgr_name
        ,sum(case when action_type = "I" then 1 else 0 end) imp
        ,sum(case when action_type = "C" then 1 else 0 end) click
        ,sum(case when action_type = "CTV1" then 1 else 0 end) ctv1
        ,sum(case when action_type = "CTV2" then 1 else 0 end) ctv2
        ,sum(case when action_type in("CTV1","CTV2") then 1 else 0 end) ctv
        ,sum(case when action_type = "VTV1" then 1 else 0 end) vtv1
        ,sum(case when action_type = "VTV2" then 1 else 0 end) vtv2
        ,sum(case when action_type in("VTV1","VTV2") then 1 else 0 end) vtv
        ,sum(winning_price)/1000000 as win_price
        ,sum(winning_price*(1+csm.margin/100))/1000000 net_sales
        ,sum(winning_price*(1+csm.margin/100)/(1-cam.margin/100))/1000000 gross_sales
        ,sum(case when xd_action_type in("xCTV1","xCTV2") then 1 else 0 end) xctv
        ,sum(case when xd_action_type in("xVTV1","xVTV2") then 1 else 0 end) xvtv
        from dsplog.adserver ads 
        inner join dm.hierarchies hie on ads.tactics_id = hie.tactics_id 
        left join agency_console.campaign cp on ads.strategy_id = cp.real_strategy_id 
        left join agency_console.campaign_agency_margin cam on cp.campaign_id = cam.campaign_id and concat_ws('-', year, month, day) between cam.start_date and cam.end_date 
        left join agency_console.campaign_smn_margin csm on cp.campaign_id = csm.campaign_id and concat_ws('-', year, month, day) between csm.start_date and csm.end_date 
        left join agency_console.advertiser ad on cp.advertiser_id = ad.advertiser_id
        left join agency_console.agency_user agu1 on ad.smn_sales_member_id = agu1.agency_user_id 
        left join agency_console.agency_user agu2 on ad.smn_operation_member_id = agu2.agency_user_id
        left join agency_console.product_ctgr ctgr on hie.product_ctgr_id = ctgr.product_ctgr_id
        left join agency_console.product_ctgr_concrete conc on hie.product_ctgr_concrete_id = conc.product_ctgr_concrete_id
        inner join xd_tmp on xd_tmp.advertiser_id = ads.advertiser_id
        where 
        concat_ws('-', year, month, day) between '{start_date}' and  '{end_date}' 
        and hie.ad_type in (0,4)
        group by 1,2,3,4,5,6,7,8,9,10,11
        order by yymmdd
        )
        select *
        from xd_tmp
        
        union all
        
        select *
        from all_tmp
        """
)
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    df_shukei2 = ir.sql_to_pandas(query)
    df_shukei2 = df_shukei2.astype({'win_price':float, 'net_sales':float, 'gross_sales':float})
    gs = GSheet()
    #クロスデバイス効果シートへ書込み2
    url = "https://docs.google.com/spreadsheets/d/1ayT6UVygczD8b1Q_XkSsdg1KskH4uCYTp_pyrSdCAZ4/edit#gid=865724583"
    sheet_index = 1
    col_offset=0
    gs.append_data(url, sheet_index, df_shukei2, col_offset)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
     

リタゲグループ登録対象アタックリストの作成

In [21]:
import datetime
import pathlib
import math

from dateutil.relativedelta import relativedelta

In [22]:
year = datetime.date.today().year
month = datetime.date.today().month
today = datetime.date.today()
this_month = today - relativedelta(months=0)
this_month = datetime.date.strftime(this_month, '%m')

In [23]:
print(this_month)

09


In [24]:
%%slack_notify @yusuke_akada
query = (
    f"""
    select 
    year
    ,month
    ,ads.advertiser_id
    ,hie.advertiser_name
    ,hie.agency_id
    ,hie.agency_name
    ,agu1.name as smn_sales_name
    ,agu2.name as smn_operation_name
    ,agu1.unit
    ,sum(win_price*(1+csm.margin/100))/1000000 net_sales
    ,sum(win_price*(1+csm.margin/100)/(1-cam.margin/100))/1000000 gross_sales
    ,max(case when rtg.name like '%XD_%' then 1 else 0 end) as rtg_f
    from dm.domain_ads ads 
    inner join dm.hierarchies hie on ads.target_id = hie.target_id 
    left join agency_console.campaign cp on ads.strategy_id = cp.real_strategy_id 
    left join agency_console.campaign_agency_margin cam on cp.campaign_id = cam.campaign_id and concat_ws('-', year, month, day) between cam.start_date and cam.end_date 
    left join agency_console.campaign_smn_margin csm on cp.campaign_id = csm.campaign_id and concat_ws('-', year, month, day) between csm.start_date and csm.end_date 
    left join agency_console.advertiser ad on cp.advertiser_id = ad.advertiser_id
    left join agency_console.agency_user agu1 on ad.smn_sales_member_id = agu1.agency_user_id 
    left join agency_console.agency_user agu2 on ad.smn_operation_member_id = agu2.agency_user_id
    left join console.retargeting_group rtg on hie.advertiser_id = rtg.advertiser_id
    where 
    year = '{year}' 
    and month = "{this_month}" 
    and ads.strategy_id not in (2, 10101)  
    AND ads.advertiser_id not in(27,2014,4958,583)
    and ad_type in (0,4)
    group by 1,2,3,4,5,6,7,8,9
    order by advertiser_id
    """
)
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    df_tmp = ir.sql_to_pandas(query)
    df_tmp = df_tmp.astype({'net_sales':float, 'gross_sales':float})

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
    s

In [25]:
df_tmp.head()

,year,month,advertiser_id,advertiser_name,agency_id,agency_name,smn_sales_name,smn_operation_name,unit,net_sales,gross_sales,rtg_f
0,2019,09,46,富士通株式会社,13,株式会社日本経済社,三吉達也,佐藤雄太郎,東日本1課,3.435487e+07,3.435487e+07,1
1,2019,09,52,株式会社リソウ,9,ソネット・メディア・ネットワークス株式会社,新川真由,三輪悠太,東日本2課,1.625439e+06,2.031799e+06,0
2,2019,09,53,スカパーJSAT株式会社,9,ソネット・メディア・ネットワークス株式会社,吉田三璃,井上倫太郎,東日本2課,2.745029e+07,3.431287e+07,1
3,2019,09,101,株式会社ブレイク・フィールド社,9,ソネット・メディア・ネットワークス株式会社,鈴木直子,岡野祐希,東日本1課,6.088561e+07,6.088561e+07,0
4,2019,09,146,株式会社エデュケーショナルネットワーク,9,ソネット・メディア・ネットワークス株式会社,草本和也,草本和也,東日本2課,8.980734e+05,8.980734e+05,0


In [26]:
#クロスデバイス・リタゲ優先導入ランク一覧
#シート上のデータ消去
for i in range(11,12):
    gs = GSheet()
    gs.clear_sheets(gsheet_url2, [i])

INFO:ailab_tools.utils.gsheet:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.utils.gsuite:created OAuth2 credentials.
INFO:googleapiclient.discovery:URL being requested: GET https://www.googleapis.com/discovery/v1/apis/sheets/v4/rest
INFO:googleapiclient.discovery:URL being requested: GET https://sheets.googleapis.com/v4/spreadsheets/1NgSpXDCRqWN_Omw8NmIl03_c5bWBynV4fhVRD_Yo7No?alt=json
INFO:googleapiclient.discovery:URL being requested: POST https://sheets.googleapis.com/v4/spreadsheets/1NgSpXDCRqWN_Omw8NmIl03_c5bWBynV4fhVRD_Yo7No/values/11_RTG%E7%99%BB%E9%8C%B2%E5%AF%BE%E8%B1%A1%E3%83%AA%E3%82%B9%E3%83%88%EF%BC%88%E6%9B%B4%E6%96%B0%E7%94%A8%EF%BC%89%21A1%3AAA2062:clear?alt=json


In [27]:
#クロスデバイス・リタゲ優先導入ランク一覧へ書込み（11_RTG登録対象リスト（更新用））
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    gs = GSheet()
    sheet_index = 11
    col_offset=0
    gs.add_data(gsheet_url2, sheet_index, df_tmp, [0, 0], True)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:ailab_tools.utils.gsheet:loa

過去分まとめて

%%slack_notify @yusuke_akada

for i in range(3,16):
    query = (
        f"""
        with xd_tmp as (
        select
        'XD' as castegory
        ,concat_ws('-',year,month,day) as yymmdd
        ,ads.advertiser_id
        ,hie.advertiser_name
        ,agu1.name as smn_sales_name
        ,agu2.name as smn_operation_name
        ,agu1.unit
        ,ads.strategy_id
        ,hie.strategy_name
        ,seg.segment_name
        ,ad_type
        ,case when ad_type = 0 then 'RTG'
        when ad_type = 1 then 'ATA'
        when ad_type = 2 then 'カスタム'
        when ad_type = 3 then '類似'
        when ad_type = 4 then 'DYC'
        else 'Unknown'
        end as ad_type_name
        ,hie.product_ctgr_id
        ,ctgr.name as product_ctgr_name
        ,hie.product_ctgr_concrete_id
        ,conc.name as product_ctgr_concrete_name
        ,sum(case when action_type = "I" then 1 else 0 end) imp
        ,sum(case when action_type = "C" then 1 else 0 end) click
        ,sum(case when action_type = "CTV1" then 1 else 0 end) ctv1
        ,sum(case when action_type = "CTV2" then 1 else 0 end) ctv2
        ,sum(case when action_type in("CTV1","CTV2") then 1 else 0 end) ctv
        ,sum(case when action_type = "VTV1" then 1 else 0 end) vtv1
        ,sum(case when action_type = "VTV2" then 1 else 0 end) vtv2
        ,sum(case when action_type in("VTV1","VTV2") then 1 else 0 end) vtv
        ,sum(winning_price)/1000000 as win_price
        ,sum(winning_price*(1+csm.margin/100))/1000000 net_sales
        ,sum(winning_price*(1+csm.margin/100)/(1-cam.margin/100))/1000000 gross_sales
        ,sum(case when xd_action_type in("xCTV1","xCTV2") then 1 else 0 end) xctv
        ,sum(case when xd_action_type in("xVTV1","xVTV2") then 1 else 0 end) xvtv
        from dsplog.adserver ads 
        inner join dm.hierarchies hie on ads.target_id = hie.target_id 
        left join agency_console.campaign cp on ads.strategy_id = cp.real_strategy_id 
        left join agency_console.campaign_agency_margin cam on cp.campaign_id = cam.campaign_id and concat_ws('-', year, month, day) between cam.start_date and cam.end_date 
        left join agency_console.campaign_smn_margin csm on cp.campaign_id = csm.campaign_id and concat_ws('-', year, month, day) between csm.start_date and csm.end_date 
        left join agency_console.advertiser ad on cp.advertiser_id = ad.advertiser_id
        left join agency_console.agency_user agu1 on ad.smn_sales_member_id = agu1.agency_user_id 
        left join agency_console.agency_user agu2 on ad.smn_operation_member_id = agu2.agency_user_id
        left join dm.segment_hierarchies seg on ads.target_id = seg.target_id
        left join agency_console.product_ctgr ctgr on hie.product_ctgr_id = ctgr.product_ctgr_id
        left join agency_console.product_ctgr_concrete conc on hie.product_ctgr_concrete_id = conc.product_ctgr_concrete_id
        where 
        concat_ws('-',year,month,day) between to_date(days_add(now(),-{i})) and to_date(days_add(now(),-{i})) 
        and (segment_name like '%XD%'
        or tactics_name like '%XD%')
        and ad_type in (0,4)
        group by 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
        order by yymmdd
        )
        ,
        all_tmp as (
        select
        'ALL' as category
        ,concat_ws('-',year,month,day) as yymmdd
        ,ads.advertiser_id
        ,hie.advertiser_name
        ,agu1.name as smn_sales_name
        ,agu2.name as smn_operation_name
        ,agu1.unit
        ,ads.strategy_id
        ,hie.strategy_name
        ,"" as segment_name
        ,hie.ad_type
        ,case when hie.ad_type = 0 then 'RTG'
        when hie.ad_type = 1 then 'ATA'
        when hie.ad_type = 2 then 'カスタム'
        when hie.ad_type = 3 then '類似'
        when hie.ad_type = 4 then 'DYC'
        else 'Unknown'
        end as ad_type_name
        ,hie.product_ctgr_id
        ,ctgr.name as product_ctgr_name
        ,hie.product_ctgr_concrete_id
        ,conc.name as product_ctgr_concrete_name
        ,sum(case when action_type = "I" then 1 else 0 end) imp
        ,sum(case when action_type = "C" then 1 else 0 end) click
        ,sum(case when action_type = "CTV1" then 1 else 0 end) ctv1
        ,sum(case when action_type = "CTV2" then 1 else 0 end) ctv2
        ,sum(case when action_type in("CTV1","CTV2") then 1 else 0 end) ctv
        ,sum(case when action_type = "VTV1" then 1 else 0 end) vtv1
        ,sum(case when action_type = "VTV2" then 1 else 0 end) vtv2
        ,sum(case when action_type in("VTV1","VTV2") then 1 else 0 end) vtv
        ,sum(winning_price)/1000000 as win_price
        ,sum(winning_price*(1+csm.margin/100))/1000000 net_sales
        ,sum(winning_price*(1+csm.margin/100)/(1-cam.margin/100))/1000000 gross_sales
        ,sum(case when xd_action_type in("xCTV1","xCTV2") then 1 else 0 end) xctv
        ,sum(case when xd_action_type in("xVTV1","xVTV2") then 1 else 0 end) xvtv
        from dsplog.adserver ads 
        inner join dm.hierarchies hie on ads.tactics_id = hie.tactics_id 
        left join agency_console.campaign cp on ads.strategy_id = cp.real_strategy_id 
        left join agency_console.campaign_agency_margin cam on cp.campaign_id = cam.campaign_id and concat_ws('-', year, month, day) between cam.start_date and cam.end_date 
        left join agency_console.campaign_smn_margin csm on cp.campaign_id = csm.campaign_id and concat_ws('-', year, month, day) between csm.start_date and csm.end_date 
        left join agency_console.advertiser ad on cp.advertiser_id = ad.advertiser_id
        left join agency_console.agency_user agu1 on ad.smn_sales_member_id = agu1.agency_user_id 
        left join agency_console.agency_user agu2 on ad.smn_operation_member_id = agu2.agency_user_id
        left join agency_console.product_ctgr ctgr on hie.product_ctgr_id = ctgr.product_ctgr_id
        left join agency_console.product_ctgr_concrete conc on hie.product_ctgr_concrete_id = conc.product_ctgr_concrete_id
        inner join xd_tmp on xd_tmp.strategy_id = ads.strategy_id
        where 
        concat_ws('-',year,month,day) between to_date(days_add(now(),-{i})) and to_date(days_add(now(),-{i})) 
        and hie.ad_type in (0,4)
        group by 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
        order by yymmdd
        )
        select *
        from xd_tmp
        
        union all
        
        select *
        from all_tmp
        """
    )
    with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                        request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
        df_shukei2 = ir.sql_to_pandas(query)
        df_shukei2 = df_shukei2.astype({'win_price':float, 'net_sales':float, 'gross_sales':float})
        gs = GSheet()
        #クロスデバイス効果2
        url = "https://docs.google.com/spreadsheets/d/1ayT6UVygczD8b1Q_XkSsdg1KskH4uCYTp_pyrSdCAZ4/edit#gid=1403416686"
        sheet_index = 1
        col_offset=0
        gs.append_data(url, sheet_index, df_shukei2, col_offset)

%%slack_notify @yusuke_akada

for i in range(1,19):
    query = (
        f"""
        with xd_tmp as (
        select
        'XD' as castegory
        ,concat_ws('-',year,month,day) as yymmdd
        ,ads.advertiser_id
        ,hie.advertiser_name
        ,agu1.name as smn_sales_name
        ,agu2.name as smn_operation_name
        ,agu1.unit
        ,ad_type
        ,case when ad_type = 0 then 'RTG'
        when ad_type = 1 then 'ATA'
        when ad_type = 2 then 'カスタム'
        when ad_type = 3 then '類似'
        when ad_type = 4 then 'DYC'
        else 'Unknown'
        end as ad_type_name
        ,hie.product_ctgr_id
        ,ctgr.name as product_ctgr_name
        ,hie.product_ctgr_concrete_id
        ,conc.name as product_ctgr_concrete_name
        ,sum(case when action_type = "I" then 1 else 0 end) imp
        ,sum(case when action_type = "C" then 1 else 0 end) click
        ,sum(case when action_type = "CTV1" then 1 else 0 end) ctv1
        ,sum(case when action_type = "CTV2" then 1 else 0 end) ctv2
        ,sum(case when action_type in("CTV1","CTV2") then 1 else 0 end) ctv
        ,sum(case when action_type = "VTV1" then 1 else 0 end) vtv1
        ,sum(case when action_type = "VTV2" then 1 else 0 end) vtv2
        ,sum(case when action_type in("VTV1","VTV2") then 1 else 0 end) vtv
        ,sum(winning_price)/1000000 as win_price
        ,sum(winning_price*(1+csm.margin/100))/1000000 net_sales
        ,sum(winning_price*(1+csm.margin/100)/(1-cam.margin/100))/1000000 gross_sales
        ,sum(case when xd_action_type in("xCTV1","xCTV2") then 1 else 0 end) xctv
        ,sum(case when xd_action_type in("xVTV1","xVTV2") then 1 else 0 end) xvtv
        from dsplog.adserver ads 
        inner join dm.hierarchies hie on ads.target_id = hie.target_id 
        left join agency_console.campaign cp on ads.strategy_id = cp.real_strategy_id 
        left join agency_console.campaign_agency_margin cam on cp.campaign_id = cam.campaign_id and concat_ws('-', year, month, day) between cam.start_date and cam.end_date 
        left join agency_console.campaign_smn_margin csm on cp.campaign_id = csm.campaign_id and concat_ws('-', year, month, day) between csm.start_date and csm.end_date 
        left join agency_console.advertiser ad on cp.advertiser_id = ad.advertiser_id
        left join agency_console.agency_user agu1 on ad.smn_sales_member_id = agu1.agency_user_id 
        left join agency_console.agency_user agu2 on ad.smn_operation_member_id = agu2.agency_user_id
        left join dm.segment_hierarchies seg on ads.target_id = seg.target_id
        left join agency_console.product_ctgr ctgr on hie.product_ctgr_id = ctgr.product_ctgr_id
        left join agency_console.product_ctgr_concrete conc on hie.product_ctgr_concrete_id = conc.product_ctgr_concrete_id
        where 
        concat_ws('-',year,month,day) between to_date(days_add(now(),-{i})) and to_date(days_add(now(),-{i})) 
        and (segment_name like '%XD%'
        or tactics_name like '%XD%')
        and hie.ad_type in (0,4)
        group by 1,2,3,4,5,6,7,8,9,10,11,12,13
        order by yymmdd
        )
        ,
        all_tmp as (
        select
        'ALL' as category
        ,concat_ws('-',year,month,day) as yymmdd
        ,ads.advertiser_id
        ,hie.advertiser_name
        ,agu1.name as smn_sales_name
        ,agu2.name as smn_operation_name
        ,agu1.unit
        ,hie.ad_type
        ,case when hie.ad_type = 0 then 'RTG'
        when hie.ad_type = 1 then 'ATA'
        when hie.ad_type = 2 then 'カスタム'
        when hie.ad_type = 3 then '類似'
        when hie.ad_type = 4 then 'DYC'
        else 'Unknown'
        end as ad_type_name
        ,hie.product_ctgr_id
        ,ctgr.name as product_ctgr_name
        ,hie.product_ctgr_concrete_id
        ,conc.name as product_ctgr_concrete_name
        ,sum(case when action_type = "I" then 1 else 0 end) imp
        ,sum(case when action_type = "C" then 1 else 0 end) click
        ,sum(case when action_type = "CTV1" then 1 else 0 end) ctv1
        ,sum(case when action_type = "CTV2" then 1 else 0 end) ctv2
        ,sum(case when action_type in("CTV1","CTV2") then 1 else 0 end) ctv
        ,sum(case when action_type = "VTV1" then 1 else 0 end) vtv1
        ,sum(case when action_type = "VTV2" then 1 else 0 end) vtv2
        ,sum(case when action_type in("VTV1","VTV2") then 1 else 0 end) vtv
        ,sum(winning_price)/1000000 as win_price
        ,sum(winning_price*(1+csm.margin/100))/1000000 net_sales
        ,sum(winning_price*(1+csm.margin/100)/(1-cam.margin/100))/1000000 gross_sales
        ,sum(case when xd_action_type in("xCTV1","xCTV2") then 1 else 0 end) xctv
        ,sum(case when xd_action_type in("xVTV1","xVTV2") then 1 else 0 end) xvtv
        from dsplog.adserver ads 
        inner join dm.hierarchies hie on ads.tactics_id = hie.tactics_id 
        left join agency_console.campaign cp on ads.strategy_id = cp.real_strategy_id 
        left join agency_console.campaign_agency_margin cam on cp.campaign_id = cam.campaign_id and concat_ws('-', year, month, day) between cam.start_date and cam.end_date 
        left join agency_console.campaign_smn_margin csm on cp.campaign_id = csm.campaign_id and concat_ws('-', year, month, day) between csm.start_date and csm.end_date 
        left join agency_console.advertiser ad on cp.advertiser_id = ad.advertiser_id
        left join agency_console.agency_user agu1 on ad.smn_sales_member_id = agu1.agency_user_id 
        left join agency_console.agency_user agu2 on ad.smn_operation_member_id = agu2.agency_user_id
        left join agency_console.product_ctgr ctgr on hie.product_ctgr_id = ctgr.product_ctgr_id
        left join agency_console.product_ctgr_concrete conc on hie.product_ctgr_concrete_id = conc.product_ctgr_concrete_id
        inner join xd_tmp on xd_tmp.advertiser_id = ads.advertiser_id
        where 
        concat_ws('-',year,month,day) between to_date(days_add(now(),-{i})) and to_date(days_add(now(),-{i})) 
        and hie.ad_type in (0,4)
        group by 1,2,3,4,5,6,7,8,9,10,11,12,13
        order by yymmdd
        )
        select *
        from xd_tmp
        
        union all
        
        select *
        from all_tmp
        """
    )
    with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                        request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
        df_shukei2 = ir.sql_to_pandas(query)
        df_shukei2 = df_shukei2.astype({'win_price':float, 'net_sales':float, 'gross_sales':float})
        gs = GSheet()
        url = "https://docs.google.com/spreadsheets/d/1ayT6UVygczD8b1Q_XkSsdg1KskH4uCYTp_pyrSdCAZ4/edit#gid=865724583"
        sheet_index = 7
        col_offset=0
        gs.append_data(url, sheet_index, df_shukei2, col_offset)

query = (
    f"""
    with tmp as (
    select
    concat_ws('-',year,month,day) as yymmdd
    ,hie.advertiser_name
    ,hie.strategy_name
	,ads.tactics_id
    ,ads.target_id
    ,segment_name
	,group_concat(segment_name) as seg_names
    ,sum(case when action_type = "I" then 1 else 0 end) imp
    from dsplog.adserver ads 
    inner join dm.hierarchies hie on ads.target_id = hie.target_id 
    left join dm.segment_hierarchies seg on ads.target_id = seg.target_id 
    where 
	concat_ws('-', year, month, day) between '2019-06-23' and  '2019-06-23'
    and (segment_name like '%XD%'
    or tactics_name like '%XD%')
	and ads.advertiser_id = 8311
    group by 1,2,3,4,5,6
    order by yymmdd
    )
    select
    yymmdd
    ,advertiser_name
    ,strategy_name
    ,tactics_id
    ,segment_name
    ,sum(imp) as imp
    from tmp
    where seg_names like '%XD%'
    group by 1,2,3,4,5
    
    """
)
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    df_tmp = ir.sql_to_pandas(query)  


df_tmp